#Human Activity Recognition using Boari´s Dataset of Linear Kinematics

First attempt of training ant¡s testing an LSTM. Code modified from original's @xieyulai's GitHub code 'LSTM-for-Human-Activity-Recognition-using-2D-Pose_Pytorch'

In [1]:
from google.colab import drive

# Do once the following
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
%matplotlib inline
import pandas as pd
import numpy as np

#### Testing small random dataset that has the same shape as our dataset

In [4]:
import pandas as pd
import numpy as np

# Column names
columns = ['subject_number', 'state', 'trial_number', 'trunk', 'knee', 'foot']

# Data
data = [
    [22, 'on', 15, 17.5, 56.9, 13.0],
    [np.nan, np.nan, np.nan, 23.5, 34.5, 14.1],
    [np.nan, np.nan, np.nan, 21.5, 24.5, 15.3],
    [np.nan, np.nan, np.nan, 20.5, 21.2, 16.1],
    [np.nan, np.nan, np.nan, 23.5, 34.5, 14.1],
    [np.nan, np.nan, np.nan, 21.5, 24.5, 15.3],
    [np.nan, np.nan, np.nan, 20.5, 21.2, 16.1],
    [22, 'on', 7.0, 17.5, 56.9, 13.0],
    [np.nan, np.nan, np.nan, 23.5, 24.5, 14.1],
    [np.nan, np.nan, np.nan, 21.0, 14.5, 12.3],
    [np.nan, np.nan, np.nan, 23.5, 34.5, 14.1],
    [np.nan, np.nan, np.nan, 21.5, 24.5, 15.3],
    [np.nan, np.nan, np.nan, 20.5, 21.2, 16.1],
    [10, 'on', 15, 17.5, 56.9, 13.0],
    [np.nan, np.nan, np.nan, 23.5, 34.5, 14.1],
    [np.nan, np.nan, np.nan, 21.5, 24.5, 15.3],
    [np.nan, np.nan, np.nan, 20.5, 21.2, 16.1],
    [10, 'off', 12, 27.5, 56.9, 13.0],
    [np.nan, np.nan, np.nan, 23.5, 34.5, 14.1],
    [np.nan, np.nan, np.nan, 21.5, 24.5, 15.3],
    [np.nan, np.nan, np.nan, 20.5, 21.2, 16.1],
    [np.nan, np.nan, np.nan, 21.5, 24.5, 15.3],
    [np.nan, np.nan, np.nan, 20.5, 21.2, 18.1],
    [np.nan, np.nan, np.nan, 2.5, 22.5, 12.3],
    [np.nan, np.nan, np.nan, 25.5, 51.2, 16.1],


]

# Create DataFrame
df = pd.DataFrame(data, columns=columns)

# Display DataFrame
#print(df)


## DataFrame de Boari

In [5]:
#df = pd.read_csv("/Users/nrgcajal/Documents/LSTM/LSTM/Practice_codes/linear_kinematics_csv.csv")

In [6]:
import os
# Specify the directory path
directory_path = '/content/gdrive/MyDrive/Colab Notebooks/linear_kinematics_csv.csv'

df = pd.read_csv(directory_path)

In [7]:
import pandas as pd
import numpy as np

# Find indices of rows where 'subject_number' is not NaN
non_nan_indices = df[df['subject_number'].notna()].index

print("Indices of rows with non-NaN values in 'subject_number':", non_nan_indices)
non_nan_indices

Indices of rows with non-NaN values in 'subject_number': Int64Index([     0,    384,    783,   1222,   1749,   2179,   2574,   2974,
              3376,   3789,
            ...
            493860, 494079, 494452, 494856, 495239, 495679, 496056, 496487,
            496767, 497106],
           dtype='int64', length=884)


Int64Index([     0,    384,    783,   1222,   1749,   2179,   2574,   2974,
              3376,   3789,
            ...
            493860, 494079, 494452, 494856, 495239, 495679, 496056, 496487,
            496767, 497106],
           dtype='int64', length=884)

In [8]:
## Inicializar la variable "a" con el número de frames por sample que se quiera.

i=0
a=150
total_trials = 0

# Initialize X_data and y_data
X_data = []
y_data = []

current_trial_state = None

while i< len(df):

    possible_window = df.iloc[i:i + a, 3:]
    #print("window: ", possible_window.shape)
    if (i+a) >= len(df)+1:

        #cogemos directamente las ultimas rows
        window_sample = df.iloc[-a:, 3:]
        X_data.append(window_sample.values)
        #print("window", window_sample)

        # El último estado debería ser el mismo que este
        y_data.append(state)
        break

    else:

        #if df.iloc[i:i+a, :1].notna().any().any(): # There are non-NaN values in the first 3 columns for rows a to b, es que hay un nuevo trial dentro de esta ventana
        #if pd.notna(df.loc[i:i+a, 'trial_number']):
        if pd.notna(df['trial_number'].loc[i:i+a]).any():
            print("Hay algun trial nuevo entre estos indices: ",i,i+a)
            # count how many non nans are there

            non_nan_count = df.iloc[i:i+a, :3].count().sum()

                # está en la primera posicion?

            if pd.notna(df.loc[i, 'trial_number']) and possible_window.shape[0]==a:
                #print("There is a new subject in the first position so this is a new trial")
                total_trials = total_trials+1
                print("i con nuevo trial en posicion: ", i)
                print("total de TRIALS es: ", total_trials)
                window_sample = possible_window

                X_data.append(window_sample.values)
                #print("window: ", window_sample)

                row = df.iloc[i]
                state = row['state']
                    #current_trial_state = state
                    # Append the state to y_data
                y_data.append(state)
                i = i + a

            else:

                    # Entonces procedemos a buscar la possicion desplazando la ventana hasta que no haya más non NaN.
                #print("There is NOT a new subject in the first position")
                buscando = True
                j = i - 1
                while buscando == True:
                    possible_window = df.iloc[j:j + a, 3:]
                    #if df.iloc[j:j+a, :1].notna().any().any():
                    if pd.notna(df['trial_number'].loc[j:j+a]).any():
                        j = j-1
                        #print("probando el if, j=", j)
                    else:
                        window_sample = possible_window
                            # Append the window sample to X_data
                        X_data.append(window_sample.values)
                        #print("window", window_sample)
                        y_data.append(state)
                        buscando = False
                        i = j + a +1
                        #print("probando el else, i= ",i)


        else: # There is not a new trial in this sample
            #print("There is NOT a new subject in the first position")
            window_sample = possible_window
            i = i + a

            # Append the window sample to X_data
            X_data.append(window_sample.values)
            #print("window", window_sample)

            # Append the state to y_data
            y_data.append(state)

        print("FULL SIZE:",np.shape(X_data))
        print("Window_FULL SIZE!", window_sample.iloc[:10,:6])
# Convert lists to numpy arrays
X_data_array = np.array(X_data)
y_data_array = np.array(y_data)

# Display the shapes of X_data and y_data
print('X_data shape:', X_data_array.shape)
print('y_data shape:', y_data_array.shape)
print("y_data_array: ", y_data_array)

Streaming output truncated to the last 5000 lines.
444970           NaN  0.55289       0.91321 -0.34135  0.591239       1.363233
FULL SIZE: (3359, 150, 136)
Window_FULL SIZE!         trial_number   com_ap  com_vertical   com_ml   clav_ap  clav_vertical
445111           NaN  0.73972       0.90824 -0.23750  0.765451       1.361484
445112           NaN  0.74117       0.90804 -0.23815  0.766993       1.361377
445113           NaN  0.74262       0.90785 -0.23884  0.768532       1.361283
445114           NaN  0.74407       0.90768 -0.23956  0.770068       1.361205
445115           NaN  0.74552       0.90753 -0.24032  0.771604       1.361147
445116           NaN  0.74697       0.90740 -0.24111  0.773140       1.361113
445117           NaN  0.74842       0.90730 -0.24193  0.774677       1.361104
445118           NaN  0.74988       0.90722 -0.24278  0.776219       1.361122
445119           NaN  0.75134       0.90717 -0.24366  0.777766       1.361170
445120           NaN  0.75281       0.90714 -

***************************************

# futuro intento de guardar el train y el test set

In [9]:
#arr = np.random.random((3, 2, 4))

# Reshape the 3D array into a 2D array
#reshaped_arr = arr.reshape(arr.shape[0], -1)
#reshaped_arr
# Save the reshaped array into a CSV file


array([[0.40672701, 0.60247332, 0.08922341, 0.89666365, 0.89437992,
        0.35017568, 0.04517735, 0.27676917],
       [0.37280928, 0.77622149, 0.04125535, 0.95328885, 0.11193383,
        0.29494388, 0.00815059, 0.36091176],
       [0.14297275, 0.37697457, 0.61774982, 0.38232266, 0.42662565,
        0.94653023, 0.67315749, 0.19805833]])

In [ ]:
#arr

array([[[0.26912134, 0.16418235, 0.30979561, 0.81813973],
        [0.40129553, 0.08936999, 0.65896539, 0.54718011]],

       [[0.14460684, 0.84271227, 0.14141465, 0.62454022],
        [0.78961919, 0.2519086 , 0.89560637, 0.79080641]],

       [[0.01989184, 0.08870235, 0.139432  , 0.58313814],
        [0.83954897, 0.95873725, 0.39282934, 0.77972521]]])

In [ ]:
## Once saved, just read the csv from X_data and y_data

In [ ]:
#X_directory_path = '/content/gdrive/MyDrive/Colab Notebooks/X_data.csv'
#y_directory_path = '/content/gdrive/MyDrive/Colab Notebooks/X_data.csv'

#X_data = pd.read_csv(X_directory_path)
#y_data = pd.read_csv(y_directory_path)


***********************************

### Encoding y_data

In [10]:
from sklearn.preprocessing import LabelEncoder

#y_data_array
# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the labels
y_data_encoded = label_encoder.fit_transform(y_data_array)
y_data_encoded

array([1, 1, 1, ..., 1, 1, 1])

### Train and Test split

In [11]:
import numpy as np
from sklearn.model_selection import train_test_split

# Assuming X_data is a list of 3D arrays and y_data is a list of labels

# Convert the list of arrays to a numpy array
X_data_array = np.array(X_data)

# Reshape X_data to (number of samples, number of features)
X_data_flat = X_data_array.reshape(X_data_array.shape[0], -1)

# Convert the list of labels to a numpy array
#y_data_array = np.array(y_data)

# Split the data
X_train_flat, X_test_flat, y_train, y_test = train_test_split(
    X_data_flat, y_data_encoded, test_size=0.2, random_state=42
)

# Reshape X_train and X_test back to 3D
X_train = X_train_flat.reshape(X_train_flat.shape[0], X_data_array.shape[1], X_data_array.shape[2])
X_test = X_test_flat.reshape(X_test_flat.shape[0], X_data_array.shape[1], X_data_array.shape[2])

# Print the shapes
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)


X_train shape: (2998, 150, 136)
X_test shape: (750, 150, 136)
y_train shape: (2998,)
y_test shape: (750,)


### Preprocess

In [12]:
import torch

In [13]:
tensor_X_test = torch.from_numpy(X_test)
print('test_data_size:',tensor_X_test.size())
tensor_y_test = torch.from_numpy(y_test)
print('test_label_size:',tensor_y_test.size())
n_data_size_test = tensor_X_test.size()[0]
print('n_data_size_test:',n_data_size_test)

tensor_X_train = torch.from_numpy(X_train)
print('train_data_size:',tensor_X_train.size())
tensor_y_train = torch.from_numpy(y_train)
print('train_label_size:',tensor_y_train.size())
n_data_size_train = tensor_X_train.size()[0]
print('n_data_size_train:',n_data_size_train)

test_data_size: torch.Size([750, 150, 136])
test_label_size: torch.Size([750])
n_data_size_test: 750
train_data_size: torch.Size([2998, 150, 136])
train_label_size: torch.Size([2998])
n_data_size_train: 2998


In [14]:
import torch.nn as nn

In [15]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


In [16]:
class LSTM(nn.Module):

    def __init__(self,input_dim,hidden_dim,output_dim,layer_num):
        super(LSTM,self).__init__()
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.lstm = torch.nn.LSTM(input_dim,hidden_dim,layer_num,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim,output_dim)
        self.bn = nn.BatchNorm1d(150)

    def forward(self,inputs):

        x = self.bn(inputs)
        lstm_out,(hn,cn) = self.lstm(x)
        out = self.fc(lstm_out[:,-1,:])
        return out

In [17]:
n_hidden = 128
n_joints = 136
n_categories = 2
n_layer = 3
rnn = LSTM(n_joints,n_hidden,n_categories,n_layer)
rnn.to(device)

LSTM(
  (lstm): LSTM(136, 128, num_layers=3, batch_first=True)
  (fc): Linear(in_features=128, out_features=2, bias=True)
  (bn): BatchNorm1d(150, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [18]:
X_train.shape

(2998, 150, 136)

In [19]:
LABELS = [1,0]

In [20]:
def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i[0].item()
    return LABELS[category_i], category_i

Getting Random Data

In [21]:
import random
def randomTrainingExampleBatch(batch_size, flag, num=-1):
    if flag == 'train':
        X = tensor_X_train
        y = tensor_y_train
        data_size = n_data_size_train
    elif flag == 'test':
        X = tensor_X_test
        y = tensor_y_test
        data_size = n_data_size_test
    if num == -1:
        ran_num = random.randint(0, data_size - batch_size)
    else:
        ran_num = num
    pose_sequence_tensor = X[ran_num:(ran_num + batch_size)]
    pose_sequence_tensor = pose_sequence_tensor
    category_tensor = y[ran_num:ran_num + batch_size]  # Remove the ',' to make it a 1D tensor
    return category_tensor.long().squeeze(), pose_sequence_tensor


#Training


In [22]:
import torch.optim as optim
import time
import math

# Assuming you have two labels
num_classes = 2
criterion = nn.CrossEntropyLoss()
learning_rate = 0.0005
optimizer = optim.SGD(rnn.parameters(), lr=learning_rate, momentum=0.9)

n_iters = 100000
print_every = 10
plot_every = 1000
batch_size = 128

# Keep track of losses for plotting
current_loss = 0
all_losses = []

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

start = time.time()

for iter in range(1, n_iters + 1):

    category_tensor, input_sequence = randomTrainingExampleBatch(batch_size, 'train')
    input_sequence = input_sequence.to(device)
    category_tensor = category_tensor.to(device)
    category_tensor = torch.squeeze(category_tensor)
    category_tensor = category_tensor.to(torch.float32)
    input_sequence = input_sequence.to(torch.float32)

    optimizer.zero_grad()

    output = rnn(input_sequence)
    category_tensor = category_tensor.long()
    loss = criterion(output, category_tensor)
    loss.backward()
    optimizer.step()

    current_loss += loss.item()

    category = LABELS[int(category_tensor[0])]
    #print("ITER: " ,iter)
    # Print iter number, loss, name and guess
    if iter % print_every == 0:
        guess, guess_i = categoryFromOutput(output)
        correct = '✓' if guess == category else '✗ (%s)' % category
        print('%d %d%% (%s) %.4f  / %s %s' % (iter, iter / n_iters * 100, timeSince(start), loss, guess, correct))

    # Add current loss avg to the list of losses
    if iter % plot_every == 0:
        all_losses.append(current_loss / plot_every)
        current_loss = 0


10 0% (0m 30s) nan  / 1 ✗ (0)
20 0% (0m 47s) nan  / 1 ✗ (0)
30 0% (1m 4s) nan  / 1 ✓
40 0% (1m 20s) nan  / 1 ✓
50 0% (1m 36s) nan  / 1 ✗ (0)
60 0% (1m 52s) nan  / 1 ✓
70 0% (2m 10s) nan  / 1 ✓
80 0% (2m 27s) nan  / 1 ✗ (0)
90 0% (2m 44s) nan  / 1 ✗ (0)
100 0% (3m 1s) nan  / 1 ✗ (0)
110 0% (3m 17s) nan  / 1 ✓
120 0% (3m 34s) nan  / 1 ✓
130 0% (3m 51s) nan  / 1 ✗ (0)
140 0% (4m 8s) nan  / 1 ✓
150 0% (4m 24s) nan  / 1 ✓
160 0% (4m 41s) nan  / 1 ✗ (0)
170 0% (4m 58s) nan  / 1 ✓
180 0% (5m 15s) nan  / 1 ✗ (0)
190 0% (5m 32s) nan  / 1 ✗ (0)
200 0% (5m 49s) nan  / 1 ✗ (0)
210 0% (6m 9s) nan  / 1 ✗ (0)
220 0% (6m 25s) nan  / 1 ✗ (0)
230 0% (6m 41s) nan  / 1 ✓
240 0% (6m 58s) nan  / 1 ✓
250 0% (7m 14s) nan  / 1 ✓
260 0% (7m 30s) nan  / 1 ✗ (0)
270 0% (7m 47s) nan  / 1 ✗ (0)
280 0% (8m 5s) nan  / 1 ✗ (0)
290 0% (8m 29s) nan  / 1 ✓
300 0% (8m 45s) nan  / 1 ✓
310 0% (9m 1s) nan  / 1 ✓
320 0% (9m 18s) nan  / 1 ✗ (0)
330 0% (9m 35s) nan  / 1 ✗ (0)
340 0% (9m 51s) nan  / 1 ✓
350 0% (10m 7s) nan  / 1 

KeyboardInterrupt: 

*************************************

# Some checks for the dtype of all the partams so the model works

In [ ]:
print(f"Input tensor device: {input_sequence.device}")
print(f"Category tensor device: {category_tensor.device}")
for param_name, param in rnn.named_parameters():
    print(f"Parameter: {param_name}, Device: {param.device}")

Input tensor device: cpu
Category tensor device: cpu
Parameter: lstm.weight_ih_l0, Device: cpu
Parameter: lstm.weight_hh_l0, Device: cpu
Parameter: lstm.bias_ih_l0, Device: cpu
Parameter: lstm.bias_hh_l0, Device: cpu
Parameter: lstm.weight_ih_l1, Device: cpu
Parameter: lstm.weight_hh_l1, Device: cpu
Parameter: lstm.bias_ih_l1, Device: cpu
Parameter: lstm.bias_hh_l1, Device: cpu
Parameter: lstm.weight_ih_l2, Device: cpu
Parameter: lstm.weight_hh_l2, Device: cpu
Parameter: lstm.bias_ih_l2, Device: cpu
Parameter: lstm.bias_hh_l2, Device: cpu
Parameter: fc.weight, Device: cpu
Parameter: fc.bias, Device: cpu
Parameter: bn.weight, Device: cpu
Parameter: bn.bias, Device: cpu


In [ ]:
for param_name, param in rnn.named_parameters():
    print(f"Parameter: {param_name}, Device: {param.device}, Dtype: {param.dtype}")

Parameter: lstm.weight_ih_l0, Device: cpu, Dtype: torch.float32
Parameter: lstm.weight_hh_l0, Device: cpu, Dtype: torch.float32
Parameter: lstm.bias_ih_l0, Device: cpu, Dtype: torch.float32
Parameter: lstm.bias_hh_l0, Device: cpu, Dtype: torch.float32
Parameter: lstm.weight_ih_l1, Device: cpu, Dtype: torch.float32
Parameter: lstm.weight_hh_l1, Device: cpu, Dtype: torch.float32
Parameter: lstm.bias_ih_l1, Device: cpu, Dtype: torch.float32
Parameter: lstm.bias_hh_l1, Device: cpu, Dtype: torch.float32
Parameter: lstm.weight_ih_l2, Device: cpu, Dtype: torch.float32
Parameter: lstm.weight_hh_l2, Device: cpu, Dtype: torch.float32
Parameter: lstm.bias_ih_l2, Device: cpu, Dtype: torch.float32
Parameter: lstm.bias_hh_l2, Device: cpu, Dtype: torch.float32
Parameter: fc.weight, Device: cpu, Dtype: torch.float32
Parameter: fc.bias, Device: cpu, Dtype: torch.float32
Parameter: bn.weight, Device: cpu, Dtype: torch.float32
Parameter: bn.bias, Device: cpu, Dtype: torch.float32


********************************

In [ ]:
torch.save(rnn.state_dict(),'lstm_6_bn.pkl')

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

plt.figure()
plt.plot(all_losses)

#Testing

In [ ]:
def test(flag):
    if flag == 'train':
        n = n_data_size_train
    elif flag == 'test':
        n = n_data_size_test

    with torch.no_grad():
        right = 0
        for i in range(n):
            category_tensor, inputs = randomTrainingExampleBatch(1,flag,i)
            category = LABELS[int(category_tensor[0])]
            inputs = inputs.to(device)
            output = rnn(inputs)
            guess, guess_i = categoryFromOutput(output)
            category_i = LABELS.index(category)
            if category_i == guess_i:
                right+=1
    print(flag,'accuracy',right/n)

In [ ]:
test('test')

In [ ]:
test('train')

In [ ]:
# Keep track of correct guesses in a confusion matrix
confusion = torch.zeros(n_categories, n_categories)
n_confusion = n_data_size_test

# Go through a bunch of examples and record which are correctly guessed
for i in range(n_confusion):
    category_tensor, inputs = randomTrainingExampleBatch(1,'test',i)
    category = LABELS[int(category_tensor[0])]
    inputs = inputs.to(device)
    output = rnn(inputs)
    guess, guess_i = categoryFromOutput(output)
    category_i = LABELS.index(category)
    confusion[category_i][guess_i] += 1

# Normalize by dividing every row by its sum
for i in range(n_categories):
    confusion[i] = confusion[i] / confusion[i].sum()

# Set up plot
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(confusion.numpy())
fig.colorbar(cax)

# Set up axes
ax.set_xticklabels([''] + LABELS, rotation=90)
ax.set_yticklabels([''] + LABELS)

# Force label at every tick
ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

# sphinx_gallery_thumbnail_number = 2
plt.show()